# Using this notebook

This notebook takes a spreadsheet with two columns, the page url and the count of page views, parses out the query parameters and saves them to a spreadsheet with information useful for determining whether to filter the query parameters in BigQuery.

The easiest way to do this is to take a preliminary dbt-GA4 BQ setup and run the following query in BigQuery:

```
SELECT  
  page_location as Page,
  sum(page_views) as Pageviews
FROM `my-warehouse.my_dataset.fct_ga4__pages` 
WHERE event_date_dt = date_sub(current_date, interval 1 day) 
group by Page
```
Save the results to Google Drive, open the results and copy the URL to the *Load spreadsheet* section below and then copy the sheet tab to the sheet variable just below the sheet URL.

When you have those two variables set select Runtime > Run all and grant this notebook Google Drive permissions when prompted.

A CSV file titled query_string_list.csv with your data will be created in the file browser of this notebook. 

Connect to Google Drive

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


Load spreadsheet

In [ ]:
import pandas as pd
wb = gc.open_by_url('')
sheet = wb.worksheet('')
data = sheet.get_all_values()
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:]
df



Split Page into Path and Query String

In [ ]:
df["qs"] = df.Page.str.split('?', n=1).str.get(1)
df = df.dropna(axis=0, how='any', subset=['qs'])
df['qs'] = df['qs'].str.split("\?|\&").apply(lambda x: list(filter(None, x)))
#add index as column to join later
df['ref'] = df.index
df

In [ ]:
#convert to dict with key equal to reference column
qs = pd.Series(df.qs.values,index=df.ref).to_dict()


print(len(qs))

In [ ]:
keys = [] #store unique keys
ikv = [] # index, key, value


#for l in qs:
#  for i in l:
#    kv = i.split("=", 1)
#    if len(kv) == 2:
#      ikv.append([pos,kv[0],kv[1]])
#      keys.append(kv[0])

for items in qs.items():
  for s in items[1]:
    kv = s.split("=", 1)
    if len(kv) == 2:
      ikv.append([items[0],kv[0],kv[1]])
      keys.append(kv[0])

keys = set(keys)
#remove empty
keys = [string for string in keys if string != ""]

print(ikv[:10])
print(len(keys))

In [ ]:
#loop through keys and match to all rows where ikv == key

#kcm = key, count, matches[]
kcm = []
for key in keys:
  count = 0
  index = 0
  values = set()
  for l in ikv:
    if l[1] == key:
      index = l[0]
      count += 1
      values.add( l[2] )
  kcm.append( [index, key, count, values] )
print(kcm[:10])

In [ ]:
# convert to dataframe and add a url sample that matches

qs_map = pd.DataFrame(data=kcm, columns=['ref','key','count','sample_values'])
qs_map = qs_map.merge( df, how='left', on=['ref'], )
qs_map

In [ ]:
qs_map.to_csv('query_string_list.csv')
